# Named Entity Recognition Part 2 — Workbook

In this lesson, we're going to learn about a text analysis method called *Named Entity Recognition* (NER). This method will help us computationally identify people, places, and things (of various kinds) in a text or collection of texts.

---

## Install spaCy and Download Language Model

If you haven't installed spaCy yet, you should uncomment and run these cells. If you've already installed spaCy and the language model, you don't need to do it again.

In [ ]:
#!pip install -U spacy
#!python -m spacy download en_core_web_sm

## Import Libraries

We're going to import `spacy` and `displacy`. We're also going to import the `Counter` module for counting people, places, and things, and the `pandas` library.

In [ ]:
import spacy
from spacy import displacy
import en_core_web_sm

from collections import Counter
import pandas as pd
pd.options.display.max_rows = 600

## Load Language Model

We need to load the language model and save it as the variable `nlp`

In [ ]:
nlp = en_core_web_sm.load()

## NER with Hand Curation

In [ ]:
filepath = "../texts/literature/The-House-on-Mango-Street-Sandra-Cisneros.txt"
text = open(filepath).read()
document = nlp(text)

### Use spaCy to identify places

In [ ]:
places = []
for named_entity in document.ents:
    if named_entity.label_ == 'LOC' or named_entity.label_ == "GPE":
        places.append(named_entity.text)
places_tally = Counter(places)
df = pd.DataFrame(places_tally.most_common(), columns=['place', 'count'])
df

### Output places to CSV file

In [ ]:
df.to_csv("spacy-places.csv", encoding="utf-8", index=False)

### Manually curate CSV file

In [ ]:
confirmed_df = pd.read_csv("confirmed-places.csv", encoding="utf-8")
confirmed_places = confirmed_df['place'].to_list()

### Match *all* named entities if they match the curated list

In [ ]:
places = []
for named_entity in document.ents:
    # Check to see if named entity matches one of the manually curated places
    if named_entity.text in confirmed_places:
        places.append(named_entity.text)
places_tally = Counter(places)
df = pd.DataFrame(places_tally.most_common(), columns=['place', 'count'])
df

## Get NER in Context

We can use spaCy to get individual sentences from documents with `document.sents`

In [ ]:
sentences = [sentence for sentence in document.sents]

Let's randomly examine sentences 101-105 in this list.

In [ ]:
sentences?

Because we can grab sentences, we can make a function that locates named entities in the context of the sentence in which they appear. 

We will loop through all the sentences, and if a keyword appears in one of the named entities, then we will display the keyword bolded and with its entity label. 

In [ ]:
from IPython.display import Markdown, display
import re

def get_ner_in_context(keyword, document):
     
    #Iterate through all the sentences in the document and pull out the text of each sentence
    for sentence in document.sents:
        for named_entity in sentence.ents:
            #Check to see if the keyword is in the sentence (and ignore capitalization by making both lowercase)
            if keyword.lower() in named_entity.text.lower():
                #Use the regex library to replace linebreaks and to make the keyword bolded, again ignoring capitalization
                sentence_text = re.sub('\n', ' ', sentence.text)
                sentence_text = re.sub(f"{named_entity.text}", f"**{named_entity.text}**", sentence_text, flags=re.IGNORECASE)

                display(Markdown(f'---\n**{named_entity.label_}**  \n{sentence_text}'))

In [ ]:
get_ner_in_context("Egypt", document)

In [ ]:
get_ner_in_context("India", document)

In [ ]:
get_ner_in_context("Mars", document)

In [ ]:
get_ner_in_context(?, document)

## Process Multiple Documents

To run spaCy on multiple documents, we will use `list(nlp.pipe(texts))`. We will test this out on NYT obituaries.

In [ ]:
import glob
from pathlib import Path
import random

Make a list of all files that end with `.txt` in the `NYT-Obituaries` directory

In [ ]:
directory = "../texts/history/NYT-Obituaries"
files = glob.glob(f"{directory}/*.txt")

Create a list of all the NYT obituary texts

In [ ]:
texts = []
for file in files:
    text = open(file, encoding='utf-8').read()
    texts.append(text)

Randomly select 50 obituaries from the list of texts

In [ ]:
texts = random.sample(texts, 50)

Process all the texts

In [ ]:
documents = list(nlp.pipe(texts))

## Get People for Multiple Documents

We can use almost the exact same workflow, but we have to add one more for lop to consider each document in the list of documents.

In [ ]:
people = []

# Consider each document in the list of processed documents
for document in documents:
    for named_entity in document.ents:
        if named_entity.label_ == 'PERSON':
            people.append(named_entity.text)

people_tally = Counter(people)

df = pd.DataFrame(people_tally.most_common(), columns=['character', 'count'])
df[:100]

## Process a Long Text

If you try to use spaCy on a long document, such as a long novel, you will likely get an error. To process a long document, we need to break up the text in some way.

In [ ]:
filepath = "../texts/literature/Pride-and-Prejudice_Jane-Austen.txt"
text = open(filepath).read()

Here we split up *Pride and Prejudice* by line breaks and make it into a long list. Then we run spaCy on this long list.

In [ ]:
chunked_text = text.split('\n')
chunked_documents = list(nlp.pipe(chunked_text))

In [ ]:
entities = []

# Consider each document in the list of processed documents
for document in chunked_documents:
    for named_entity in document.ents:
        if named_entity.label_ == 'PERSON':
            entities.append(named_entity.text)

entity_tally = Counter(entities)

df = pd.DataFrame(entity_tally.most_common(), columns=['entity', 'count'])
df

## Named Entities

|Type Label|Description|
|:---:|:---:|
|PERSON|People, including fictional.|
|NORP|Nationalities or religious or political groups.|
|FAC|Buildings, airports, highways, bridges, etc.|
|ORG|Companies, agencies, institutions, etc.|
|GPE|Countries, cities, states.|
|LOC|Non-GPE locations, mountain ranges, bodies of water.|
|PRODUCT|Objects, vehicles, foods, etc. (Not services.)|
|EVENT|Named hurricanes, battles, wars, sports events, etc.|
|WORK_OF_ART|Titles of books, songs, etc.|
|LAW|Named documents made into laws.|
|LANGUAGE|Any named language.|
|DATE|Absolute or relative dates or periods.|
|TIME|Times smaller than a day.|
|PERCENT|Percentage, including ”%“.|
|MONEY|Monetary values, including unit.|
|QUANTITY|Measurements, as of weight or distance.|
|ORDINAL|“first”, “second”, etc.|
|CARDINAL|Numerals that do not fall under another type.|


## Discussion

- How well does spaCy's NER seem to be performing?
- What does it do well or not so well?
- How could you imagine researchers, data scientists, yourself using NER in a project?